###  Upload this notebook in google colab 
https://colab.research.google.com/?utm_source=scs-index

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

### Orange diseases  Classification model Training 
The classification model use axelerate Keras-based framework for AI on the edge to quickly set up model training.
After training the model is converted to tflite and kmodel formats.

More info of model used to training:  
https://github.com/AIWintermuteAI/aXeleRate

In [ ]:
!pip uninstall -y imgaug && pip uninstall -y albumentations && pip install imgaug==0.4
!git clone https://github.com/AIWintermuteAI/aXeleRate.git
import sys
sys.path.append('/content/aXeleRate')
from axelerate import setup_training, setup_inference

### Upload dataset in google drive
- 1) https://www.kaggle.com/jonathansilva2020/orange-diseases-dataset/download
- 2) extract .rar and upload folder "dataset" to google drive 

In [ ]:
# visualize our classification validation dataset with visualize_dataset function.
from axelerate.networks.common_utils.augment import visualize_classification_dataset
visualize_classification_dataset('/content/drive/MyDrive/dataset/test', num_imgs=10, img_size=224, augment=True)

Next step is defining a config dictionary.
- Type is model frontend: Classifier.
- Architecture is model backend (feature extractor): MobileNet7_5 
- Input size: 224
- Fully_connected is number of neurons in classification layers as list: 100,50
- Actual_epoch is number of epochs to train: 10
- Dropout value is dropout in classification layers: 0.5

This config convert trained model to formats with Edge TPU K210 and tflite.

In [ ]:
config = {
    "model" : {
        "type":                 "Classifier",
        "architecture":         "MobileNet7_5",
        "input_size":           224,
        "fully-connected":      [100,50],
        "labels":               [],
        "dropout" : 		0.5
    },
     "weights" : {
            "full":   				"",
            "backend":   		    "imagenet",
            "save_bottleneck":      False
        
    },
    "train" : {
        "actual_epoch":         10,
        "train_image_folder":    "/content/drive/MyDrive/dataset/train", 
        "train_times":          1,
        "valid_image_folder":   "/content/drive/MyDrive/dataset/test",
        "valid_times":          1,
        "valid_metric":         "val_accuracy",
        "batch_size":           32,
        "learning_rate":        1e-3,
        "saved_folder":   		    "",
        "first_trainable_layer": "",
        "augmentation":				True
    },
    "converter" : {
        "type":   					["k210","tflite"]
    }
}

In [ ]:
from keras import backend as K 
K.clear_session()
model_path = setup_training(config_dict=config)